In [2]:
using Oceananigans 
using Oceananigans.Units
using Oceananigans.TurbulenceClosures
using Oceananigans.Architectures: GPU
using CairoMakie 
using NCDatasets
using Printf

In [3]:
# Model parameters
Nx = 300
Nz = 80
f = 1e-4               # Coriolis frequency [s⁻¹]
L_front = 10kilometers  # Initial front width [m]
aspect_ratio = 100      # L_front/H
Ro = 0.1           # Rossby number (defines M^2)

# Derived parameters
H_front = L_front/aspect_ratio
M² = (Ro^2*f^2*L_front)/H_front
Δb = M²*L_front
κh = 1e-4
κv = 1e-4

filename = "periodic_geostrophic_adjustment"

"periodic_geostrophic_adjustment"

In [4]:

grid = RectilinearGrid(GPU(),size = (Nx, Nz), 
                       x = (-L_front/2, L_front/2),
                       z = (-H_front, 0),
                       topology = (Periodic, Flat, Bounded))

300×1×80 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CUDAGPU with 3×0×3 halo
├── Periodic x ∈ [-5000.0, 5000.0) regularly spaced with Δx=33.3333
├── Flat y                         
└── Bounded  z ∈ [-100.0, 0.0]     regularly spaced with Δz=1.25

In [5]:
horizontal_closure = HorizontalScalarDiffusivity(ν=κh, κ=κh )
vertical_closure = VerticalScalarDiffusivity(ν=κv , κ=κv )
closure = (horizontal_closure, vertical_closure)

(HorizontalScalarDiffusivity{ExplicitTimeDiscretization}(ν=0.0001, κ=0.0001), VerticalScalarDiffusivity{ExplicitTimeDiscretization}(ν=0.0001, κ=0.0001))

In [6]:
model =  NonhydrostaticModel(; grid,
                coriolis = FPlane(f = f),
                buoyancy = BuoyancyTracer(),
                tracers = (:b,:T),
                advection = WENO(),
                closure = closure)

NonhydrostaticModel{GPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── grid: 300×1×80 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CUDAGPU with 3×0×3 halo
├── timestepper: RungeKutta3TimeStepper
├── advection scheme: WENO{3, Float64, Float32}(order=5)
├── tracers: (b, T)
├── closure: Tuple with 2 closures:
│   ├── HorizontalScalarDiffusivity{ExplicitTimeDiscretization}(ν=0.0001, κ=(b=0.0001, T=0.0001))
│   └── VerticalScalarDiffusivity{ExplicitTimeDiscretization}(ν=0.0001, κ=(b=0.0001, T=0.0001))
├── buoyancy: BuoyancyTracer with ĝ = NegativeZDirection()
└── coriolis: FPlane{Float64}(f=0.0001)

In [7]:
bᵢ(x, z) = Δb*sin(2*pi/L_front * x)
Tᵢ(x, z) = exp(-(x/(L_front/50)).^2)

set!(model, b= bᵢ, T= Tᵢ, u = 0, v = 0, w = 0)  # Start from rest

In [8]:
simulation = Simulation(model, Δt=20minutes, stop_time=4days)


Simulation of NonhydrostaticModel{GPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 20 minutes
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN days
├── Stop time: 4 days
├── Stop iteration: Inf
├── Wall time limit: Inf
├── Minimum relative step: 0.0
├── Callbacks: OrderedDict with 4 entries:
│   ├── stop_time_exceeded => 4
│   ├── stop_iteration_exceeded => -
│   ├── wall_time_limit_exceeded => e
│   └── nan_checker => }
├── Output writers: OrderedDict with no entries
└── Diagnostics: OrderedDict with no entries

In [9]:
conjure_time_step_wizard!(simulation, IterationInterval(20), cfl=0.2, max_Δt=20minutes)

In [10]:

wall_clock = Ref(time_ns())

function print_progress(sim)
    u, v, w = model.velocities
    progress = 100 * (time(sim) / sim.stop_time)
    elapsed = (time_ns() - wall_clock[]) / 1e9

    @printf("[%05.2f%%] i: %d, t: %s, wall time: %s, max(u): (%6.3e, %6.3e, %6.3e) m/s, next Δt: %s\n",
            progress, iteration(sim), prettytime(sim), prettytime(elapsed),
            maximum(abs, u), maximum(abs, v), maximum(abs, w), prettytime(sim.Δt))

    wall_clock[] = time_ns()

    return nothing
end

add_callback!(simulation, print_progress, IterationInterval(50))

In [11]:
# Output setup
u, v, w = model.velocities
b = model.tracers.b
wc = Field(@at (Center, Center, Center) model.velocities.w)
T = model.tracers.T

#For Lagrangian filtering
simulation.output_writers[:jld2fields] = JLD2Writer(
    model, (; b, u, v, w, wc,T), filename=filename * ".jld2", schedule=TimeInterval(1hour), overwrite_existing=true)


#for python visualisation
rm(filename * ".nc",force=true)
simulation.output_writers[:ncfields] = NetCDFWriter(
    model, (; b, u, v, w, wc,T), filename=filename * ".nc", schedule=TimeInterval(1hour), overwrite_existing=true)
    

NetCDFWriter scheduled on TimeInterval(1 hour):
├── filepath: periodic_geostrophic_adjustment.nc
├── dimensions: time(0), x_faa(300), x_caa(300), z_aaf(81), z_aac(80)
├── 6 outputs: (v, w, T, wc, b, u)
├── array_type: Array{Float32}
├── file_splitting: NoFileSplitting
└── file size: 27.5 KiB

In [12]:
@info "Running the simulation..."

run!(simulation)

@info "Simulation completed in " * prettytime(simulation.run_wall_time)

[ Info: Running the simulation...
[ Info: Initializing simulation...


[00.00%] i: 0, t: 0 seconds, wall time: 28.868 seconds, max(u): (0.000e+00, 0.000e+00, 0.000e+00) m/s, next Δt: 20 minutes


[ Info:     ... simulation initialization complete (17.400 seconds)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (2.730 seconds).


[11.91%] i: 50, t: 11.437 hours, wall time: 5.180 seconds, max(u): (2.375e-02, 4.848e-02, 4.424e-04) m/s, next Δt: 8.744 minutes
[16.27%] i: 100, t: 15.616 hours, wall time: 197.389 ms, max(u): (1.921e-02, 8.735e-03, 3.065e-04) m/s, next Δt: 4.517 minutes
[20.27%] i: 150, t: 19.455 hours, wall time: 201.433 ms, max(u): (1.913e-02, 8.346e-03, 3.684e-04) m/s, next Δt: 5.465 minutes
[23.96%] i: 200, t: 23 hours, wall time: 182.074 ms, max(u): (2.704e-02, 4.190e-02, 5.152e-04) m/s, next Δt: 4.107 minutes
[27.69%] i: 250, t: 1.107 days, wall time: 198.404 ms, max(u): (6.064e-03, 5.944e-02, 1.007e-04) m/s, next Δt: 4.969 minutes
[31.89%] i: 300, t: 1.276 days, wall time: 201.692 ms, max(u): (2.810e-02, 3.286e-02, 6.305e-04) m/s, next Δt: 3.953 minutes
[35.42%] i: 350, t: 1.417 days, wall time: 178.163 ms, max(u): (1.247e-02, 4.980e-03, 1.848e-04) m/s, next Δt: 4.741 minutes
[39.78%] i: 400, t: 1.591 days, wall time: 220.612 ms, max(u): (2.580e-02, 1.879e-02, 5.998e-04) m/s, next Δt: 4.305 mi

[ Info: Simulation is stopping after running for 25.418 seconds.
[ Info: Simulation time 4 days equals or exceeds stop time 4 days.
[ Info: Simulation completed in 25.436 seconds
